# colab

In [ ]:
# 파일 확인

import os

dataset_dir = "/content/drive/MyDrive/Colab Notebooks/Training/ResNet_DOG"

if not os.path.exists(dataset_dir):
    print("❌ 데이터셋 폴더가 존재하지 않습니다!")
else:
    print("✅ 데이터셋 폴더 확인 완료!")
    print("클래스 폴더:", os.listdir(dataset_dir))


for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):  # 클래스 폴더인지 확인
        print(f"🔹 {class_name} 폴더 내 이미지 개수: {len(os.listdir(class_path))}")

## No Optuna

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision import models
import os

# Google Drive 연동 (필요한 경우)
from google.colab import drive
drive.mount('/content/drive')

def train_and_test():
    # ------------------------
    # 1. 경로 및 매개변수 설정
    # ------------------------
    dataset_dir = "/content/drive/MyDrive/Colab Notebooks/Training/ResNet_DOG"  # 코랩 로컬 파일 경로
    batch_size = 16   # 16, 32, 64
    num_epochs = 30   # 5, 10, 20, 30, 50
    learning_rate = 0.0001   # 0.1, 0,01, 0.001(1e-3), 0.0005, 0.0001(1e-4), 0.00001
    log_file = "/content/drive/MyDrive/Colab Notebooks/Training/training_log.txt"  # 로그 파일 저장 경로

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    print("Using device:", device)

    # 로그 파일 초기화
    with open(log_file, "w") as f:
        f.write(f"Training Log\n")
        f.write(f"Batch Size: {batch_size}\n")
        f.write(f"Epochs: {num_epochs}\n")
        f.write(f"Learning Rate: {learning_rate}\n")
        f.write(f"Device: {device}\n\n")

    # ------------------------
    # 2. 데이터 변환 설정
    # ------------------------
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # ------------------------
    # 3. 데이터셋 로드 및 분할
    # ------------------------
    full_dataset = torchvision.datasets.ImageFolder(root=dataset_dir, transform=transform)
    total_images = len(full_dataset)

    train_size = int(0.7 * total_images)
    val_size = int(0.15 * total_images)
    test_size = total_images - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

    print(f"Train: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")

    with open(log_file, "a") as f:
        f.write(f"Train Samples: {len(train_dataset)}\n")
        f.write(f"Validation Samples: {len(val_dataset)}\n")
        f.write(f"Test Samples: {len(test_dataset)}\n\n")

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    num_classes = len(full_dataset.classes)
    print("Classes:", full_dataset.classes)

    # ------------------------
    # 4. ResNet 모델 정의 (간략한 모델 정보 로그)
    # ------------------------
    model_name = "ResNet-18"  # 변경 가능 (예: "ResNet-50" 등)
    model = models.resnet18(weights="IMAGENET1K_V1")  # ResNet-50 사용 시 models.resnet50()

    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    model.to(device)

    # ------------------------
    # 5. 손실 및 옵티마이저
    # ------------------------
    criterion = nn.CrossEntropyLoss()
    weight_decay = 1e-4
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)   # AdamW 적용

    with open(log_file, "a") as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"Criterion: {criterion}\n")
        f.write(f"Optimizer: {optimizer}\n\n")

    # ------------------------
    # 6. 훈련 루프
    # ------------------------
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = 100.0 * correct / total

        log_msg = (f"Epoch [{epoch+1}/{num_epochs}] "
                   f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.2f}% | "
                   f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        print(log_msg)

        # 로그 파일 저장
        with open(log_file, "a") as f:
            f.write(log_msg + "\n")

    # ------------------------
    # 7. 테스트
    # ------------------------
    model.eval()
    test_correct, test_total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            test_correct += predicted.eq(labels).sum().item()
            test_total += labels.size(0)
    test_acc = 100.0 * test_correct / test_total
    test_msg = f"Test Accuracy: {test_acc:.2f}%"
    print(test_msg)

    # 테스트 결과 로그 저장
    with open(log_file, "a") as f:
        f.write("\n" + test_msg + "\n")

    # ------------------------
    # 8. 모델 저장
    # ------------------------
    model_save_path = "/content/drive/MyDrive/Colab Notebooks/Training/resnet18_balanced_model.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved at {model_save_path}")

    with open(log_file, "a") as f:
        f.write(f"\nModel saved at: {model_save_path}\n")
        f.write("Training and testing complete!\n")
        

if __name__ == '__main__':
    train_and_test()

## Optuna (최적의 파라미터 찾기)

In [ ]:
pip install optuna

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision import models
import os
import optuna  # Optuna 라이브러리
import numpy as np


# Google Drive 연동 (필요한 경우)
from google.colab import drive
drive.mount('/content/drive')


# ------------------------
# 1. 데이터셋 설정
# ------------------------
dataset_dir = "/content/drive/MyDrive/Colab Notebooks/Training/ResNet_DOG"  # 폴더 안에 클래스별 이미지 존재

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")   # colab
# device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print("✅ Using device:", device)


def train_model(trial):
    """
    Optuna가 최적의 하이퍼파라미터를 찾기 위한 목적 함수
    """

    # 1) 최적의 Batch Size 찾기
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])

    # 데이터 로더 생성 (num_workers=8, pin_memory=True 설정)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=12, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=12, pin_memory=True)

    # 2) ResNet50 모델 생성
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    model.to(device)

    # 3) 최적의 Optimizer 찾기 (Adam vs AdamW vs SGD)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW", "SGD"])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)  # 최적의 learning rate 탐색
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-2)  # 최적의 weight decay 탐색

    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=weight_decay)

    # 손실 함수 정의
    criterion = nn.CrossEntropyLoss()

    # 혼합 정밀도 학습 (FP16 활용)
    scaler = torch.amp.GradScaler()

    # 4) 모델 학습 (5 Epoch만 실행, Optuna에서 빠른 탐색을 위해)
    num_epochs = 15
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            with torch.autocast(device_type="cuda"):  # Mixed Precision Training     # mps
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = 100.0 * correct / total

        # 5) 검증 데이터에서 성능 평가
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                with torch.autocast(device_type="cuda"):    # mps 
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = 100.0 * val_correct / val_total

        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f}, Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

    return val_loss  # 검증 데이터 손실값이 최소가 되는 조합을 찾음


# ------------------------
# 6. 메인 실행 (멀티프로세싱 해결)
# ------------------------
if __name__ == "__main__":

    # 데이터셋 로드 및 분할
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    full_dataset = torchvision.datasets.ImageFolder(root=dataset_dir, transform=transform)
    total_images = len(full_dataset)

    # Train/Validation/Test Split (70:15:15)
    train_size = int(0.7 * total_images)
    val_size = int(0.15 * total_images)
    test_size = total_images - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

    num_classes = len(full_dataset.classes)
    print(f"🔹 Train: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")
    print("🔹 Classes:", full_dataset.classes)

    # Optuna 실행 (20회 탐색)
    study = optuna.create_study(direction="minimize")  # 최소의 val_loss를 찾는 방향
    study.optimize(train_model, n_trials=20)  # 20회 탐색

    # 최적의 하이퍼파라미터 출력
    best_params = study.best_params
    print("\n✅ 최적의 하이퍼파라미터 찾기 완료!")
    print(best_params)

    # ------------------------
    # 7. 최적의 하이퍼파라미터를 사용하여 모델 재훈련
    # ------------------------
    batch_size = best_params["batch_size"]
    learning_rate = best_params["learning_rate"]
    weight_decay = best_params["weight_decay"]
    optimizer_name = best_params["optimizer"]

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)
    model.to(device)

    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=weight_decay)

    print("🚀 최적의 하이퍼파라미터로 모델을 재학습하세요!")